# Coursera Capstone Project

## The Battle of Neighborhoods (Week 1)


### Durgesh Kumar Singh - Part 1 : Introduction and Data Sections

In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.




### Coursera Capstone - REPORT CONTENT

1.	Introduction Section : 
	⁃	Discussion of the business problem and the interested audience in this project.
2.	Data Section:  
	⁃	 Description of the data that will be used to solve the problem and the sources.
3.	Methodology section 
	⁃	Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4.	Results section 
	⁃	Discussion of the results.
5.	Discussion section 
	⁃	Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6.	Conclusion section 
	⁃	Report Conclusion.

In [19]:
# A description of the problem and a discussion of the background. (15 marks)

# 1.	Introduction Section : 
### Discussion of the business problem and the audience who would be interested in this project.


### Description of the Problem and Background

### Scenario:

I am a data scientist residing in Karol Bagh, New Delhi. 
I currently live within walking distance to Karol Bagh Metro Station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment.
I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen.
The key question is :  How can I find a convenient and enjoyable place similar to mine now in Singapore?
Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course.
In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:
 - apartment must be 2 or 3 bedrooms
 - desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius
 - price of rent not exceed $7,000 per month
 - top ammenities in the selected neighborhood shall be similar to current residence
 - desirable to have venues such as coffee shops, restaurants, wine stores, gym and food shops
 - as a reference, I have included a map of venues near current residence in Singapore.
    

### Business Problem:   
The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.


### Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.


In [20]:
# A description of the data and how it will be used to solve the problem. (15 marks)

# 2. Data Section:
### Description of the data and its sources that will be used to solve the problem

### Description of the Data:

The following data is required to  answer the issues of the problem:

- List of Boroughs and neighborhoods of Manhattan with their geodata (latitude and longitude) 
- List of Subway metro stations in Manhattan with their address location 
- List of apartments for rent in Manhattan area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
- Venues for each Manhattan neighborhood ( that can be clustered)
- Venues for subway metro stations, as needed


### How the data will be used to solve the problem

The data will be used as follows:
- Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)
- Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
- create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.


The procesing of these DATA will allow to answer the key questions to make a decision:
- what is the cost of rent (per square ft) around a mile radius from each subway metro station?
- what is the area of Manhattan with best rental pricing that meets criteria established?
- What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Manhattan neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.



### Reference of venues around current residence in Singapore for comparison to Manhattan place

In [14]:
# Old Rajendra Nagar, Karol Bagh, New Delhi
geolocator=Nominatim(user_agent="myGeocoder")
location=geolocator.geocode("Karol Bagh, New Delhi")
print("Latitude={}, Longitude={}".format(location.latitude, location.longitude))

Latitude=28.6529982, Longitude=77.1890227


In [15]:
neighborhood_latitude=28.6529982
neighborhood_longitude=77.1890227

In [16]:
CLIENT_ID = '1CRRCS2YEW1I4ZPCIP4JBTFUNTEQLX53F0MPXXXHGNGUQ1CR' # your Foursquare ID
CLIENT_SECRET = 'GZDF2LIEBAVEZ5PWQ4V5UCOCDDIUAHXNEKNAI2DQ121HPNXU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1CRRCS2YEW1I4ZPCIP4JBTFUNTEQLX53F0MPXXXHGNGUQ1CR
CLIENT_SECRET:GZDF2LIEBAVEZ5PWQ4V5UCOCDDIUAHXNEKNAI2DQ121HPNXU


In [17]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1CRRCS2YEW1I4ZPCIP4JBTFUNTEQLX53F0MPXXXHGNGUQ1CR&client_secret=GZDF2LIEBAVEZ5PWQ4V5UCOCDDIUAHXNEKNAI2DQ121HPNXU&v=20180604&ll=28.6529982,77.1890227&radius=500&limit=100'

In [18]:
results = requests.get(url).json()
#results

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

<ipython-input-20-ef7be79cd008>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  SGnearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Raviraj Ki Kulfi,Dessert Shop,28.649359,77.190215
1,Roshan Di Kulfi | रोशन दी क़ुल्फ़ी,Snack Place,28.650428,77.192391
2,Florence Hotel,Hotel,28.652200,77.191703
3,Ashok International Hotell,Hotel,28.653310,77.191591
4,Kodak express,Camera Store,28.657403,77.188514


## Map of Delhi with venues near residence place - for reference 

In [27]:
# create map of DELHI place  using latitude and longitude values
map_sg = folium.Map(location=[28.6529982, 77.1890227], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg

In [29]:
# Thanks !!

In [30]:
## 3.	Methodology section 


In [31]:
## 4.	Results section 


In [32]:
## 5.	Discussion section 


In [33]:
## 6.	Conclusion section 
